<a href="https://colab.research.google.com/github/RadhaGulhane13/PyTorch-Labs/blob/master/lab0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import PyTorch and Torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import time

In [ ]:
# Download training data from open datasets.
training_data = datasets.SVHN(
    root="data",
    split="train",
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/182040794 [00:00<?, ?it/s]

In [ ]:
# Download test data from open datasets.
test_data = datasets.SVHN(
    root="data",
    split="test",
    download=True,
    transform=ToTensor(),
)

batch_size = 64


  0%|          | 0/64275384 [00:00<?, ?it/s]

In [ ]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
# Define model
model = torchvision.models.resnet50(pretrained=True)
#print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    t_start = time.time()
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print(f"Time taken:{time.time()-t_start:>3f}")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 7.164317  [    0/73257]
loss: 2.546369  [ 6400/73257]
loss: 2.424540  [12800/73257]
loss: 2.534268  [19200/73257]
loss: 2.489761  [25600/73257]
loss: 2.278239  [32000/73257]
loss: 2.300957  [38400/73257]
loss: 2.303333  [44800/73257]
loss: 2.388859  [51200/73257]
loss: 2.244948  [57600/73257]
loss: 2.413825  [64000/73257]
loss: 2.358060  [70400/73257]
Time taken:1686.291826
Test Error: 
 Accuracy: 16.2%, Avg loss: 2.383426 

Epoch 2
-------------------------------
loss: 2.319007  [    0/73257]
loss: 2.194271  [ 6400/73257]
loss: 2.293240  [12800/73257]
loss: 2.315545  [19200/73257]
loss: 2.314415  [25600/73257]
loss: 2.287921  [32000/73257]
loss: 2.298405  [38400/73257]
loss: 2.248081  [44800/73257]
loss: 2.318192  [51200/73257]
loss: 2.237313  [57600/73257]
loss: 2.378895  [64000/73257]
